In [ ]:
#data source: https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020

In [ ]:
#!pip install --upgrade pip

In [ ]:
#!pip install nltk

In [ ]:
#!pip install --force-reinstall numpy==1.17

In [ ]:
#!pip install enum34

In [ ]:
#!pip install sentence_transformers --ignore-installed PyYAML

In [ ]:
#!pip install ipywidgets

In [1]:
import pandas as pd
import numpy as np

import itertools
import re

from tqdm import tqdm

import nltk
from nltk import sent_tokenize

from sentence_transformers import SentenceTransformer
import pickle

In [2]:
#nltk.download('punkt')

In [3]:
sent_detector = nltk.data.load('/home/dmitry-ds/nltk_data/tokenizers/punkt/english.pickle')

In [4]:
transformer_model = SentenceTransformer('all-MiniLM-L12-v2')

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
anime_s = pd.read_csv('/home/dmitry-ds/rec-sys/data/anime_with_synopsis.csv')

In [7]:
anime_s.head()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


In [8]:
anime_s.shape

(16214, 5)

In [9]:
anime_selected = pd.read_csv('data/anime_selected.csv')

/opt/conda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,25,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
anime_s.rename(columns = {'MAL_ID': 'anime_id'}, inplace = True)

In [17]:
anime_selected.shape

(14353, 81)

## Get anime synopsis embeddings

In [18]:
anime_s = anime_selected.merge(anime_s, on = 'anime_id', how = 'left')

In [19]:
anime_s

,anime_id,Name,Score,Genres,English name_x,Japanese name_x,Type_x,Episodes_x,Aired_x,Premiered_x,Producers_x,Licensors_x,Studios_x,Source_x,Duration_x,Rating_x,Ranked_x,Popularity_x,Members_x,Favorites_x,Watching_x,Completed_x,On-Hold_x,Dropped_x,Plan to Watch_x,Score-10_x,Score-9_x,Score-8_x,Score-7_x,Score-6_x,Score-5_x,Score-4_x,Score-3_x,Score-2_x,Score-1_x,Genres_sep_x,Action_x,Adventure_x,Cars_x,Comedy_x,Dementia_x,Demons_x,Drama_x,Ecchi_x,Fantasy_x,Game_x,Harem_x,Hentai_x,Historical_x,Horror_x,Josei_x,Kids_x,Magic_x,Martial Arts_x,Mecha_x,Military_x,Music_x,Mystery_x,Parody_x,Police_x,Psychological_x,Romance_x,Samurai_x,School_x,Sci-Fi_x,Seinen_x,Shoujo_x,Shoujo Ai_x,Shounen_x,Shounen Ai_x,Slice of Life_x,Space_x,Sports_x,Super Power_x,Supernatural_x,Thriller_x,Unknown_x,Vampire_x,Yaoi_x,Yuri_x,Start air_x,Name_x,Score_x,Genres_x,English name_y,Japanese name_y,Type_y,Episodes_y,Aired_y,Premiered_y,Producers_y,Licensors_y,Studios_y,Source_y,Duration_y,Rating_y,Ranked_y,Popularity_y,Members_y,Favorites_y,Watching_y,Completed_y,On-Hold_y,Dropped_y,Plan to Watch_y,Score-10_y,Score-9_y,Score-8_y,Score-7_y,Score-6_y,Score-5_y,Score-4_y,Score-3_y,Score-2_y,Score-1_y,Genres_sep_y,Action_y,Adventure_y,Cars_y,Comedy_y,Dementia_y,Demons_y,Drama_y,Ecchi_y,Fantasy_y,Game_y,Harem_y,Hentai_y,Historical_y,Horror_y,Josei_y,Kids_y,Magic_y,Martial Arts_y,Mecha_y,Military_y,Music_y,Mystery_y,Parody_y,Police_y,Psychological_y,Romance_y,Samurai_y,School_y,Sci-Fi_y,Seinen_y,Shoujo_y,Shoujo Ai_y,Shounen_y,Shounen Ai_y,Slice of Life_y,Space_y,Sports_y,Super Power_y,Supernatural_y,Thriller_y,Unknown_y,Vampire_y,Yaoi_y,Yuri_y,Start air_y,Name_y,Score_y,Genres_y,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),28.0,39,1251960,61971,105808,718161,71513,26678,329800,229170,182126.0,131625.0,62330,20688,8904,3184.0,1357.0,741.0,1580.0,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,90s,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),28.0,39.0,1251960.0,61971.0,105808.0,718161.0,71513.0,26678.0,329800.0,229170,182126.0,131625.0,62330,20688,8904,3184.0,1357.0,741.0,1580.0,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90s,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518,273145,1174,4143,208333,1935,770,57964,30043,49201.0,49505.0,22632,5805,1877,577.0,221.0,109.0,379.0,"['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']",1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,00s,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518.0,273145.0,1174.0,4143.0,208333.0,1935.0,770.0,57964.0,30043,49201.0,49505.0,22632,5805,1877,577.0,221.0,109.0,379.0,"['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']",1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.

In [20]:
anime_s['sypnopsis'].fillna('No synopsis found for this series', inplace=True)

In [22]:
anime_s_tokenized = anime_s['sypnopsis'].apply(sent_detector.tokenize)

In [23]:
anime_s_tokenized[0]

['In the year 2071, humanity has colonized several of the planets and moons of the solar system leaving the now uninhabitable surface of planet Earth behind.',
 'The Inter Solar System Police attempts to keep peace in the galaxy, aided in part by outlaw bounty hunters, referred to as "Cowboys."',
 'The ragtag team aboard the spaceship Bebop are two such individuals.',
 'Mellow and carefree Spike Spiegel is balanced by his boisterous, pragmatic partner Jet Black as the pair makes a living chasing bounties and collecting rewards.',
 "Thrown off course by the addition of new members that they meet in their travels—Ein, a genetically engineered, highly intelligent Welsh Corgi; femme fatale Faye Valentine, an enigmatic trickster with memory loss; and the strange computer whiz kid Edward Wong—the crew embarks on thrilling adventures that unravel each member's dark and mysterious past little by little.",
 'Well-balanced with high density action and light-hearted comedy, Cowboy Bebop is a spac

In [24]:
anime_s_emb = []

for i in tqdm(anime_s_tokenized):
    if len(i) > 1:
        anime_s_emb.append(np.squeeze(np.mean(transformer_model.encode(i), axis = 0)))
    else:
        anime_s_emb.append(np.squeeze(transformer_model.encode(i)))

100%|██████████| 14353/14353 [07:57<00:00, 30.04it/s]


In [25]:
anime_s['synopsis_emb'] = anime_s_emb

In [28]:
anime_s.head()

,anime_id,Name,Score,Genres,English name_x,Japanese name_x,Type_x,Episodes_x,Aired_x,Premiered_x,Producers_x,Licensors_x,Studios_x,Source_x,Duration_x,Rating_x,Ranked_x,Popularity_x,Members_x,Favorites_x,Watching_x,Completed_x,On-Hold_x,Dropped_x,Plan to Watch_x,Score-10_x,Score-9_x,Score-8_x,Score-7_x,Score-6_x,Score-5_x,Score-4_x,Score-3_x,Score-2_x,Score-1_x,Genres_sep_x,Action_x,Adventure_x,Cars_x,Comedy_x,Dementia_x,Demons_x,Drama_x,Ecchi_x,Fantasy_x,Game_x,Harem_x,Hentai_x,Historical_x,Horror_x,Josei_x,Kids_x,Magic_x,Martial Arts_x,Mecha_x,Military_x,Music_x,Mystery_x,Parody_x,Police_x,Psychological_x,Romance_x,Samurai_x,School_x,Sci-Fi_x,Seinen_x,Shoujo_x,Shoujo Ai_x,Shounen_x,Shounen Ai_x,Slice of Life_x,Space_x,Sports_x,Super Power_x,Supernatural_x,Thriller_x,Unknown_x,Vampire_x,Yaoi_x,Yuri_x,Start air_x,Name_x,Score_x,Genres_x,English name_y,Japanese name_y,Type_y,Episodes_y,Aired_y,Premiered_y,Producers_y,Licensors_y,Studios_y,Source_y,Duration_y,Rating_y,Ranked_y,Popularity_y,Members_y,Favorites_y,Watching_y,Completed_y,On-Hold_y,Dropped_y,Plan to Watch_y,Score-10_y,Score-9_y,Score-8_y,Score-7_y,Score-6_y,Score-5_y,Score-4_y,Score-3_y,Score-2_y,Score-1_y,Genres_sep_y,Action_y,Adventure_y,Cars_y,Comedy_y,Dementia_y,Demons_y,Drama_y,Ecchi_y,Fantasy_y,Game_y,Harem_y,Hentai_y,Historical_y,Horror_y,Josei_y,Kids_y,Magic_y,Martial Arts_y,Mecha_y,Military_y,Music_y,Mystery_y,Parody_y,Police_y,Psychological_y,Romance_y,Samurai_y,School_y,Sci-Fi_y,Seinen_y,Shoujo_y,Shoujo Ai_y,Shounen_y,Shounen Ai_y,Slice of Life_y,Space_y,Sports_y,Super Power_y,Supernatural_y,Thriller_y,Unknown_y,Vampire_y,Yaoi_y,Yuri_y,Start air_y,Name_y,Score_y,Genres_y,sypnopsis,synopsis_emb
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),28.0,39,1251960,61971,105808,718161,71513,26678,329800,229170,182126.0,131625.0,62330,20688,8904,3184.0,1357.0,741.0,1580.0,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,90s,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),28.0,39.0,1251960.0,61971.0,105808.0,718161.0,71513.0,26678.0,329800.0,229170,182126.0,131625.0,62330,20688,8904,3184.0,1357.0,741.0,1580.0,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90s,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever...","[-0.03795919, 0.003579858, -0.01721148, -0.026..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518,273145,1174,4143,208333,1935,770,57964,30043,49201.0,49505.0,22632,5805,1877,577.0,221.0,109.0,379.0,"['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']",1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,00s,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518.0,273145.0,1174.0,4143.0,208333.0,1935.0,770.0,57964.0,30043,49201.0,49505.0,22632,5805,1877,577.0,221.0,109.0,379.0,"['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']",1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0

In [29]:
emb_columns = ['emb'+str(i) for i in range(384)]

In [30]:
anime_s_copy = anime_s.synopsis_emb.apply(pd.Series)

In [31]:
anime_s_copy.columns = emb_columns

In [33]:
anime_s_copy.insert(loc = 0, column = 'anime_id', value = anime_s['anime_id'])

In [34]:
anime_s_copy.head()

,anime_id,emb0,emb1,emb2,emb3,emb4,emb5,emb6,emb7,emb8,emb9,emb10,emb11,emb12,emb13,emb14,emb15,emb16,emb17,emb18,emb19,emb20,emb21,emb22,emb23,emb24,emb25,emb26,emb27,emb28,emb29,emb30,emb31,emb32,emb33,emb34,emb35,emb36,emb37,emb38,emb39,emb40,emb41,emb42,emb43,emb44,emb45,emb46,emb47,emb48,emb49,emb50,emb51,emb52,emb53,emb54,emb55,emb56,emb57,emb58,emb59,emb60,emb61,emb62,emb63,emb64,emb65,emb66,emb67,emb68,emb69,emb70,emb71,emb72,emb73,emb74,emb75,emb76,emb77,emb78,emb79,emb80,emb81,emb82,emb83,emb84,emb85,emb86,emb87,emb88,emb89,emb90,emb91,emb92,emb93,emb94,emb95,emb96,emb97,emb98,emb99,emb100,emb101,emb102,emb103,emb104,emb105,emb106,emb107,emb108,emb109,emb110,emb111,emb112,emb113,emb114,emb115,emb116,emb117,emb118,emb119,emb120,emb121,emb122,emb123,emb124,emb125,emb126,emb127,emb128,emb129,emb130,emb131,emb132,emb133,emb134,emb135,emb136,emb137,emb138,emb139,emb140,emb141,emb142,emb143,emb144,emb145,emb146,emb147,emb148,emb149,emb150,emb151,emb152,emb153,emb154,emb155,emb156,emb157,emb158,emb159,emb160,emb161,emb162,emb163,emb164,emb165,emb166,emb167,emb168,emb169,emb170,emb171,emb172,emb173,emb174,emb175,emb176,emb177,emb178,emb179,emb180,emb181,emb182,emb183,emb184,emb185,emb186,emb187,emb188,emb189,emb190,emb191,emb192,emb193,emb194,emb195,emb196,emb197,emb198,emb199,emb200,emb201,emb202,emb203,emb204,emb205,emb206,emb207,emb208,emb209,emb210,emb211,emb212,emb213,emb214,emb215,emb216,emb217,emb218,emb219,emb220,emb221,emb222,emb223,emb224,emb225,emb226,emb227,emb228,emb229,emb230,emb231,emb232,emb233,emb234,emb235,emb236,emb237,emb238,emb239,emb240,emb241,emb242,emb243,emb244,emb245,emb246,emb247,emb248,emb249,emb250,emb251,emb252,emb253,emb254,emb255,emb256,emb257,emb258,emb259,emb260,emb261,emb262,emb263,emb264,emb265,emb266,emb267,emb268,emb269,emb270,emb271,emb272,emb273,emb274,emb275,emb276,emb277,emb278,emb279,emb280,emb281,emb282,emb283,emb284,emb285,emb286,emb287,emb288,emb289,emb290,emb291,emb292,emb293,emb294,emb295,emb296,emb297,emb298,emb299,emb300,emb301,emb302,emb303,emb304,emb305,emb306,emb307,emb308,emb309,emb310,emb311,emb312,emb313,emb314,emb315,emb316,emb317,emb318,emb319,emb320,emb321,emb322,emb323,emb324,emb325,emb326,emb327,emb328,emb329,emb330,emb331,emb332,emb333,emb334,emb335,emb336,emb337,emb338,emb339,emb340,emb341,emb342,emb343,emb344,emb345,emb346,emb347,emb348,emb349,emb350,emb351,emb352,emb353,emb354,emb355,emb356,emb357,emb358,emb359,emb360,emb361,emb362,emb363,emb364,emb365,emb366,emb367,emb368,emb369,emb370,emb371,emb372,emb373,emb374,emb375,emb376,emb377,emb378,emb379,emb380,emb381,emb382,emb383
0,1,-0.037959,0.003580,-0.017211,-0.026130,0.019456,-0.019987,0.022723,0.017017,-0.028059,-0.007092,0.012273,0.008578,0.013832,-0.006762,-0.042466,0.034519,0.004619,-0.014714,0.019883,0.016247,-0.050487,-0.004393,-0.008996,0.030360,-0.061954,-0.003686,0.004443,0.035934,-0.009892,-0.010420,-0.007989,0.024069,-0.003625,-0.000580,0.024303,0.060942,0.007046,0.016987,-0.006914,-0.007529,0.011035,-0.051239,0.037628,0.021332,-0.010090,-0.040340,-0.036259,-0.006836,-0.017065,0.002348,0.015120,-0.063076,0.010560,-0.032255,0.017137,-0.059919,0.000420,0.014235,-0.014135,-0.043292,-0.007693,-0.020321,0.002422,0.018317,0.006145,-0.013301,-0.009228,0.014055,-0.020301,-0.025373,0.002097,0.003052,-0.004086,0.021696,0.007335,0.051250,0.020829,0.000427,-0.016291,-0.038543,0.024526,-0.027519,-0.041594,-0.024013,-0.007039,-0.022419,0.005931,-0.006159,0.009990,0.031608,-0.056912,0.019725,0.076017,-0.003284,-0.016677,0.045867,0.035525,-0.031557,-0.011693,0.066680,0.020487,-0.014247,-0.002489,0.013377,-0.023016,-0.039304,0.031196,-0.006577,0.041985,-0.061005,-0.034111,0.008960,-0.010846,0.046408,0.040058,-0.004427,0.007366,0.054954,-0.071297,-0.029499,0.069899,0.045632,0.021967,0.012959,0.003102,-0.017090,-0.028539,-0.015805,0.016950,-0.034482,-0.015134,0.000362,0.024712,-0.009196,-0.036359,0.014683,-0.040213,0.025584,-0.007407,-0.034642,0.032497,0.027439,-0.005449,-0.026949,-0.051447,-0.031572,0.017978,-0.01402

In [35]:
#anime_s_copy.to_csv('/home/dmitry-ds/rec-sys/data/anime_synopsis_emb_2.csv', index = False)